# Basic CSGO analysis
#### Last Updated: June 15, 2020
The csgo package was developed with easy analysis in mind. To that end, the data parsed goes directly into Pandas dataframes, as shown in the first example notebook, [Parsing a CSGO demofile](https://github.com/pnxenopoulos/csgo/blob/master/examples/00_Parsing_a_CSGO_demofile.ipynb). Such a format allows for easy analysis, especially when using basic aggregate statistics.

### What statistics are common in CSGO?
CSGO uses a variety of statistics we can calculate from the demofile data. Some basic statistics are:

- Kill/Death Ratio (KDR), defined as $\frac{Kills}{Deaths}$.
- Average Damage per Round (ADR), defined as $\frac{Damage}{Rounds}$. 
- Headshot Percentage, defined as $\frac{Headshot Kills}{Kills}$.
- Utility damage, which just represents the amount of damage a player inflicts using incendiary, molotiv and HE grenades.

These are just a few basic statistics that can easily be calculate using our data. To start, we reference the same demofile from [ESL Pro League Season 10 Finals](https://www.hltv.org/matches/2338065/astralis-vs-liquid-esl-pro-league-season-10-finals), where we look at the first map of the series, `de_inferno`.

In [1]:
from csgo.parser import DemoParser

# Create parser object
# Set log=True above if you want to produce a logfile for the parser
demo_parser = DemoParser(demofile = "astralis-vs-liquid-m1-inferno.dem", match_id = "astralis-vs-liquid-m1-inferno.dem")


# Parse the demofile, output results to dictionary with df name as key
data = demo_parser.parse()

11:01:43 [INFO] Initialized CSGODemoParser with demofile astralis-vs-liquid-m1-inferno.dem
11:01:43 [INFO] Go version>=1.14.0
11:01:43 [INFO] Starting CSGO Golang demofile parser, reading in /home/peter/Downloads/examples/astralis-vs-liquid-m1-inferno.dem
11:01:43 [INFO] Running Golang parser from /home/peter/.pyenv/versions/3.8.3/lib/python3.8/site-packages/csgo-0.1-py3.8.egg/csgo/parser/
11:01:59 [INFO] Demofile parsing complete
11:01:59 [INFO] Parsing match events
11:01:59 [INFO] Parsed round end 1
11:01:59 [INFO] Parsed round end official 1
11:01:59 [INFO] Parsed round end 2
11:01:59 [INFO] Parsed round end official 2
11:01:59 [INFO] Parsed round end 3
11:01:59 [INFO] Parsed round end official 3
11:01:59 [INFO] Parsed round end 4
11:01:59 [INFO] Parsed round end official 4
11:01:59 [INFO] Parsed round end 5
11:01:59 [INFO] Parsed round end official 5
11:01:59 [INFO] Parsed round end 6
11:01:59 [INFO] Parsed round end official 6
11:01:59 [INFO] Parsed round end 7
11:01:59 [INFO] Par

## KDR
KDR can be calculated through simple aggregations. In the `Kills` dataframe, accessed via `data["Kills"]`, we can simply tabulate the number of times a player appeared as the _Attacker_ (kills) and the _Victim_ (deaths).

In [2]:
kills = data["Kills"].groupby(["AttackerName"]).size().reset_index(name="Kills")
deaths = data["Kills"].groupby(["VictimName"]).size().reset_index(name="Deaths")
kdr = kills.merge(deaths, left_on = "AttackerName", right_on = "VictimName")
kdr["KDR"] = kdr["Kills"]/kdr["Deaths"]
kdr = kdr[["AttackerName", "Kills", "Deaths", "KDR"]]
kdr.columns = ["PlayerName", "Kills", "Deaths", "KDR"]
kdr.sort_values(by=["KDR"], ascending=False)

,PlayerName,Kills,Deaths,KDR
0,EliGE,26,16,1.625000
8,gla1ve,21,16,1.312500
2,NAF,20,17,1.176471
5,Xyp9x,20,17,1.176471
3,Stewie2K,25,25,1.000000
9,nitr0,18,19,0.947368
6,device,18,20,0.900000
7,dupreeh,18,23,0.782609
4,Twistzz,11,16,0.687500
1,Magisk,16,24,0.666667


## ADR
ADR can be calculate simply by taking the total damage produced by a player divided by the number of rounds. In CSGO, a player can inflict healthpoint (HP) or armor damage. Additionally, we provide the raw damage output in the column `HpDamage`, since weapons like the AWP can inflict damages of over 100, along with the normalized damages in `KillHpDamage`, which have a maximum value of 100, which is a player's total HP.

First, we find the total number of rounds played by checking the number of rows in `data["Rounds"]`. Then, for each damage entry, we sum the Hp and Armor damages. Then, we sum the total damage for each player and divide by the total number of rounds.

In [3]:
# Find total number of rounds
total_rounds = data["Rounds"].shape[0]

# Add Armor damage to the HP damage for each damage entry to get total damage
data["Damages"]["HpDamageArmor"] = data["Damages"]["HpDamage"] + data["Damages"]["ArmorDamage"]
data["Damages"]["KillHpDamageArmor"] = data["Damages"]["KillHpDamage"] + data["Damages"]["ArmorDamage"]

# Calculate
adr = (data["Damages"].groupby(["AttackerName"])["HpDamageArmor", "KillHpDamageArmor"].sum()/total_rounds).reset_index()
adr.columns = ["PlayerName", "RawADR", "NormADR"]
adr.sort_values(by=["RawADR"], ascending=False)

<ipython-input-3-ad2419c50648>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  adr = (data["Damages"].groupby(["AttackerName"])["HpDamageArmor", "KillHpDamageArmor"].sum()/total_rounds).reset_index()


,PlayerName,RawADR,NormADR
4,Stewie2K,131.066667,127.900000
0,EliGE,117.266667,107.500000
9,gla1ve,104.766667,102.633333
3,NAF,104.200000,99.366667
8,dupreeh,102.466667,93.766667
1,Magisk,97.333333,93.066667
6,Xyp9x,90.966667,87.266667
10,nitr0,89.833333,85.766667
7,device,77.400000,72.733333
5,Twistzz,69.633333,67.900000


## Headshoot Percentage
Headshots can be a measure of a player's aiming accuracy. In the kills dataframe (`data["Kills"]`), we indicate whether a kill was a headshot in the `col` column.

In [4]:
data["Kills"].groupby("AttackerName").IsHeadshot.mean().reset_index(name="HeadShotPct").sort_values("HeadShotPct", ascending=False)

,AttackerName,HeadShotPct
7,dupreeh,0.666667
3,Stewie2K,0.480000
5,Xyp9x,0.450000
0,EliGE,0.384615
4,Twistzz,0.363636
8,gla1ve,0.333333
1,Magisk,0.312500
2,NAF,0.300000
6,device,0.277778
9,nitr0,0.222222


We can also break down each player's headshot percentage by the weapon type. We filter by player-weapon combos that had at minimum 5 kills.

In [5]:
headshots = data["Kills"].groupby(["AttackerName", "Weapon"]).IsHeadshot.agg(["mean", "count"]).reset_index()
headshots = headshots[headshots["count"] > 5]
headshots.sort_values("mean", ascending=False)

,AttackerName,Weapon,mean,count
48,dupreeh,AK47,0.666667,9
59,gla1ve,SG556,0.571429,7
38,Xyp9x,SG556,0.538462,13
3,EliGE,M4A4,0.500000,6
4,EliGE,SG556,0.500000,8
24,Stewie2K,M4A4,0.400000,10
9,Magisk,M4A4,0.285714,7
15,NAF,M4A4,0.166667,6
0,EliGE,AK47,0.125000,8
13,NAF,AWP,0.000000,7


## Utility Damage
Utility damage, inflicted by grenades such as the incendiary grenade (on CT), molotov (on T) and HE grenade, can be another measure of player skill. While smokes and flash grenades can injure opponents (while this is rare, it exists in our selected data!) we do not count these damage events as utility damage. We can access grenade damage events in the damages dataframe (`data["Damages"]`), and the grenade events in the grenades dataframe (`data["Grenades"`).

In [6]:
data["Damages"].Weapon.unique()

array(['Glock', 'USP', 'HE', 'Molotov', 'Deagle', 'AK47', 'Galil',
       'SG556', 'World', 'CZ', 'MP9', 'M4A4', 'Incendiary', 'Bomb',
       'Mac10', 'AWP', 'P2000', 'Famas', 'FiveSeven', 'P250', 'UMP',
       'Knife', 'Smoke'], dtype=object)

In [7]:
nade_dmg = data["Damages"][data["Damages"]["Weapon"].isin(["Incendiary", "Molotov", "HE"])]

In [8]:
nade_dmg_df = nade_dmg.groupby("AttackerName").HpDamageArmor.sum().reset_index(name="UtilityDamage").sort_values("UtilityDamage", ascending=False)
nade_dmg_df.columns = ["PlayerName", "UtilityDamage"]
nade_dmg_df

,PlayerName,UtilityDamage
3,Stewie2K,369
8,gla1ve,347
1,Magisk,261
6,device,261
0,EliGE,233
9,nitr0,230
2,NAF,180
5,Xyp9x,114
4,Twistzz,108
7,dupreeh,33


We can also break down the utilty damage by the associated grenade.

In [9]:
nade_dmg.groupby(["AttackerName", "Weapon"]).HpDamageArmor.sum().reset_index(name="UtilityDamage").sort_values("UtilityDamage", ascending=False)

,AttackerName,Weapon,UtilityDamage
21,gla1ve,HE,281
17,device,HE,210
3,Magisk,HE,200
9,Stewie2K,HE,140
8,NAF,Molotov,136
1,EliGE,Incendiary,130
10,Stewie2K,Incendiary,127
25,nitr0,Incendiary,120
14,Xyp9x,HE,109
24,nitr0,HE,102


To find out how many grenades a player threw, we can access the grenades dataframe in `data["Grenades"]`.

In [10]:
nades_thrown_df = data["Grenades"][data["Grenades"]["GrenadeType"].isin(["HE", "Incendiary"])].groupby("PlayerName").size().reset_index(name="NadesThrown")
nades_thrown_df

,PlayerName,NadesThrown
0,EliGE,22
1,Magisk,25
2,NAF,14
3,Stewie2K,22
4,Twistzz,21
5,Xyp9x,13
6,device,20
7,dupreeh,12
8,gla1ve,27
9,nitr0,17


Lastly, we can combine `nade_dmg_df` and `nades_thrown_df` to create a dmg-per-nade metric.

In [11]:
nade_df = nade_dmg_df.merge(nades_thrown_df, on = "PlayerName")
nade_df["UDperNade"] = nade_df["UtilityDamage"]/nade_df["NadesThrown"]
nade_df.sort_values("UDperNade", ascending=False)

,PlayerName,UtilityDamage,NadesThrown,UDperNade
0,Stewie2K,369,22,16.772727
5,nitr0,230,17,13.529412
3,device,261,20,13.050000
6,NAF,180,14,12.857143
1,gla1ve,347,27,12.851852
4,EliGE,233,22,10.590909
2,Magisk,261,25,10.440000
7,Xyp9x,114,13,8.769231
8,Twistzz,108,21,5.142857
9,dupreeh,33,12,2.750000
